In [4]:
import requests
import googlemaps
from datetime import datetime
import polyline
import json
import pandas as pd
import numpy as np
import geopandas
import fiona
import re
import random

from config import MAP_QUEST_KEY
from config import GMAPS_KEY

In [5]:
MapQuestKey = MAP_QUEST_KEY
GMapsKey = GMAPS_KEY 

In [15]:
def liveStats(routes):
    l=[]
    a=polyline.decode(routes[0]['overview_polyline']['points'])
    for j in range(len(a)):
        l.append(a[j][0])
        l.append(a[j][1])
    r = requests.get("http://open.mapquestapi.com/elevation/v1/profile?key="+MapQuestKey+"&shapeFormat=raw&latLngCollection="+str(l)[1:-1])
    elevation = pd.DataFrame((json.loads(r.content))['elevationProfile'])
    roc = [0]
    for i in range(len(elevation)-1):
        roc.append((elevation["height"][i+1] - elevation["height"][i])/(elevation["distance"][i+1] - elevation["distance"][i]))
    elevation["roc"]=roc
    asc=0
    desc=0
    ascList = []
    descList = []
    distList = []
    for i in range(len(elevation)):
        if (elevation["roc"][i]<0):
            desc += (elevation["distance"][i] - elevation["distance"][i-1])
            ascList.append(asc)
            descList.append(desc)
            distList.append(elevation["distance"][i])
        elif (elevation["roc"][i]>0):
            asc += (elevation["distance"][i] - elevation["distance"][i-1])
            ascList.append(asc)
            descList.append(desc)
            distList.append(elevation["distance"][i])
    df = pd.DataFrame()
    df["Distance"] = distList
    df["Ascent"] = ascList
    df["Descent"] = descList
    df["Flat"] = df["Distance"] - df["Ascent"] - df["Descent"]
    return df

In [16]:
gmaps = googlemaps.Client(key=GMapsKey)
now = datetime.now()
routes = gmaps.directions([1.353428, 103.682113], [1.340360, 103.682757],mode="driving",departure_time=now,alternatives= False)
res = liveStats(routes)

In [17]:
res

,Distance,Ascent,Descent,Flat
0,0.0817,0.0000,0.0817,0.0000
1,0.2694,0.1332,0.0817,0.0545
2,0.3798,0.1332,0.1921,0.0545
3,0.4174,0.1487,0.1921,0.0766
4,0.4957,0.1487,0.2444,0.1026
5,0.5775,0.1487,0.2812,0.1476
6,0.6199,0.1911,0.2812,0.1476
7,0.6386,0.1911,0.2999,0.1476
8,0.7492,0.2367,0.2999,0.2126
9,0.8068,0.2367,0.3328,0.2373
